***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/GermanCreditData.csv')
df['Credit_type'] = df['Credit_type'].map({1: 0, 2: 1})

encoder = OrdinalEncoder()
df['Status'] = encoder.fit_transform(df[['Status']])
df['Credit_history'] = encoder.fit_transform(df[['Credit_history']])
df['Purpose'] = encoder.fit_transform(df[['Purpose']])
df['Savings_account'] = encoder.fit_transform(df[['Savings_account']])
df['Present_employment'] = encoder.fit_transform(df[['Present_employment']])
df['Personal_status'] = encoder.fit_transform(df[['Personal_status']])
df['Other_debtors'] = encoder.fit_transform(df[['Other_debtors']])
df['Property'] = encoder.fit_transform(df[['Property']])
df['Other_installment'] = encoder.fit_transform(df[['Other_installment']])
df['Housing'] = encoder.fit_transform(df[['Housing']])
df['Job'] = encoder.fit_transform(df[['Job']])
df['Telephone'] = encoder.fit_transform(df[['Telephone']])
df['foreign_worker'] = encoder.fit_transform(df[['foreign_worker']])

df = df.dropna()

X = df.loc[:, df.columns != 'Credit_type']
y = df['Credit_type']

rus = RandomOverSampler(random_state=42)
X, y = rus.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)
cv = StratifiedKFold(n_splits=10)

grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

In [ ]:
best = LinearSVC(C=1, random_state=42)
best.fit(X, y)

cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')

print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
train_predictions = best.predict(X)

train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.42691605 0.41782356 0.58199538 0.40813008 0.60227273 0.42208462
 0.57522494 0.60529169 0.41782356 0.48263119]
Mean CV Accuracy: 0.4940193802776425
Training Set Accuracy: 0.5185714285714286


##**Extracting weights and intercept values**

In [ ]:
b = best.intercept_
w = best.coef_
w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['Credit_type'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'Credit_Type_Predicted': predicted_label})
index_class = results_df.index[results_df['Credit_Type_Predicted'] == 1]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.3 MB/s eta 0:00:00
  Created wheel for z3: filename=z3-0.2.0-py3-none-any.whl size=26629 sha256=75f05d3e7f4bd262984a6509779ceacb97e39d274b2c2adead98ef010268122e
  Stored in directory: /root/.cache/pip/wheels/5a/b2/60/55b07a5084cad7ab411e395fb1440a2b1a19598bff535a3955
Successfully built z3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.7 MB/s eta 0:00:00


##**Optimizer -- step 1**

In [ ]:
from z3 import *
import csv
import time

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = og_x_np[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    constraint = And(X[1] >= 7, X[1] <= 35)
    opt.add(constraint)

    opt.add(Or(X[2] == 0, X[2] == 1, X[2] == 2, X[2] == 3, X[2] == 4))

    original_value3 = og_x_np[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    constraint1 = And(X[4] >= 1500, X[4] <= 8500)
    opt.add(constraint1)

    original_value5 = og_x_np[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = og_x_np[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = og_x_np[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    original_value8 = og_x_np[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    original_value9 = og_x_np[9]
    if original_value9 is not None:
        opt.add(X[9] == original_value9)

    original_value10 = og_x_np[10]
    if original_value10 is not None:
        opt.add(X[10] == original_value10)

    original_value11 = og_x_np[11]
    if original_value11 is not None:
        opt.add(X[11] == original_value11)

    original_value12 = og_x_np[12]
    if original_value12 is not None:
        opt.add(X[12] == original_value12)

    original_value13 = og_x_np[13]
    if original_value13 is not None:
        opt.add(X[13] == original_value13)

    original_value14 = og_x_np[14]
    if original_value14 is not None:
        opt.add(X[14] == original_value14)

    original_value15 = og_x_np[15]
    if original_value15 is not None:
        opt.add(X[15] == original_value15)

    constraint5 = Or(X[16] == 0, X[16] == 1, X[16] == 2, X[16] == 3)
    opt.add(constraint5)

    constraint6 = Or(X[17] == 1, X[17] == 2)
    opt.add(constraint6)

    original_value18 = og_x_np[18]
    if original_value18 is not None:
        opt.add(X[18] == original_value18)

    original_value19 = og_x_np[19]
    if original_value19 is not None:
        opt.add(X[19] == original_value19)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B < 0
    opt.add(inequality)

    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time() 
    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)

    end_time = time.time() 
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step1.csv', names=column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

[0]
[0]
[0]


##**Optimizer -- step 2**

In [ ]:
def step2(W, B, row):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    slack_duration = Real("slack_duration")
    slack_duration_constr = (slack_duration >= 0)
    constraint = And(X[1] > 0.5, X[1] >= 7 - slack_duration, X[1] <= 35 + slack_duration)
    opt.add(constraint, slack_duration_constr)

    opt.add(Or(X[2] == 0, X[2] == 1, X[2] == 2, X[2] == 3, X[2] == 4))

    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    slack_credit = Real("slack_credit")
    slack_credit_constr = (slack_credit >= 0)
    constraint1 = And(X[4] > 0, X[4] >= 1500 - slack_credit, X[4] <= 8500 + slack_credit)
    opt.add(constraint1, slack_credit_constr)

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = row[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    original_value8 = row[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    original_value9 = row[9]
    if original_value9 is not None:
        opt.add(X[9] == original_value9)

    original_value10 = row[10]
    if original_value10 is not None:
        opt.add(X[10] == original_value10)

    original_value11 = row[11]
    if original_value11 is not None:
        opt.add(X[11] == original_value11)

    original_value12 = row[12]
    if original_value12 is not None:
        opt.add(X[12] == original_value12)

    original_value13 = row[13]
    if original_value13 is not None:
        opt.add(X[13] == original_value13)

    original_value14 = row[14]
    if original_value14 is not None:
        opt.add(X[14] == original_value14)

    original_value15 = row[15]
    if original_value15 is not None:
        opt.add(X[15] == original_value15)

    constraint5 = Or(X[16] == 0, X[16] == 1, X[16] == 2, X[16] == 3)
    opt.add(constraint5)

    constraint6 = Or(X[17] == 1, X[17] == 2)
    opt.add(constraint6)

    original_value18 = row[18]
    if original_value18 is not None:
        opt.add(X[18] == original_value18)

    original_value19 = row[19]
    if original_value19 is not None:
        opt.add(X[19] == original_value19)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B < 0
    opt.add(inequality)

    objective = (slack_duration + slack_credit)
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)

        # Calculate the optimized cost
        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        slack_duration = model.eval(slack_duration).as_decimal(15)
        print(f"slack_duration: {slack_duration}")

        slack_credit = model.eval(slack_credit).as_decimal(15)
        print(f"slack_credit: {slack_credit}")


        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_duration, slack_credit
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '']

W = w
B = b[0]
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, cost, cost_optimized, slack_duration, slack_credit = step2(W, B, row)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_duration] + [slack_credit]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_duration', 'Slack_credit']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 3**

In [ ]:
def step3(W, B, row, slack_value1, slack_value2):

    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    constraint = And(X[1] > 0.5, X[1] >= 7 - slack_value1, X[1] <= 35 + slack_value1)
    opt.add(constraint)

    opt.add(Or(X[2] == 0, X[2] == 1, X[2] == 2, X[2] == 3, X[2] == 4))

    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    constraint1 = And(X[4] > 0, X[4] >= 1500 - slack_value2, X[4] <= 8500 + slack_value2)
    opt.add(constraint1)

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = row[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    original_value8 = row[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    original_value9 = row[9]
    if original_value9 is not None:
        opt.add(X[9] == original_value9)

    original_value10 = row[10]
    if original_value10 is not None:
        opt.add(X[10] == original_value10)

    original_value11 = row[11]
    if original_value11 is not None:
        opt.add(X[11] == original_value11)

    original_value12 = row[12]
    if original_value12 is not None:
        opt.add(X[12] == original_value12)

    original_value13 = row[13]
    if original_value13 is not None:
        opt.add(X[13] == original_value13)

    original_value14 = row[14]
    if original_value14 is not None:
        opt.add(X[14] == original_value14)

    original_value15 = row[15]
    if original_value15 is not None:
        opt.add(X[15] == original_value15)

    constraint5 = Or(X[16] == 0, X[16] == 1, X[16] == 2, X[16] == 3)
    opt.add(constraint5)

    constraint6 = Or(X[17] == 1, X[17] == 2)
    opt.add(constraint6)

    original_value18 = row[18]
    if original_value18 is not None:
        opt.add(X[18] == original_value18)

    original_value19 = row[19]
    if original_value19 is not None:
        opt.add(X[19] == original_value19)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B < 0
    opt.add(inequality)

    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_duration'].iloc[i]
    slack_value2 = last['Slack_credit'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2)

    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

Optimized x': ['0', '0.506772710444386?', '4', '4', '1234.3367970975441', '4', '4', '4', '2', '0', '4', '0', '67', '2', '1', '2', '0', '1', '1', '0']
Optimized distance: 3
Optimized cost: 3
Optimized x': ['1', '0.506772710444386?', '4', '4', '781.9854058978331', '0', '2', '2', '1', '0', '2', '0', '22', '2', '1', '1', '0', '1', '0', '0']
Optimized distance: 4
Optimized cost: 4
Optimized x': ['3', '0.506772710444386?', '4', '7', '1456.1233683188204', '0', '3', '2', '2', '0', '3', '0', '49', '2', '1', '1', '0', '1', '0', '0']
Optimized distance: 4
Optimized cost: 4
Optimized x': ['0', '0.506772710444386?', '4', '3', '832.1012168855559', '0', '3', '2', '2', '2', '4', '1', '45', '2', '2', '1', '0', '1', '0', '0']
Optimized distance: 5
Optimized cost: 5
Optimized x': ['0', '0.506772710444386?', '4', '0', '541.1764946090925', '0', '2', '3', '2', '0', '4', '3', '53', '2', '2', '2', '0', '1', '0', '0']
Optimized distance: 5
Optimized cost: 5
Optimized x': ['3', '0.506772710444386?', '4', '7', '

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0 & l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step1_optimized.csv')

d = pd.concat([d1, d3_initial])

l1_runtime = d.loc[:, 'Runtime'].mean()
print("L1 runtime", l1_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

99.69199178644764


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/GermanCredit/New-Step2.csv')
number_of_slacks_per_record = 2

count_s1 = 0
count_s2 = 0
total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_duration'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['Slack_duration'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_credit'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_credit'].iloc[i] == 0:
        count_s2 = 0

    total_count = count_s1 + count_s2
    # print(count_s1, count_s2)
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100
print(average_per_dataset)

98.40370751802266
